In [ ]:
# @title Instalar dependencias

r = "Retrieval"
b = "based"
v = "Voice"
c = "Conversion"
w = "WebUI"

!git clone https://github.com/RVC-Project/{"-".join([r,b,v,c,w])}
!mv {"-".join([r,b,v,c,w])} rvc
%cd rvc
!rm {"_".join([r,b,v,c,w])}*.ipynb

!pip install -q -r requirements.txt
!apt install -qq aria2

# Descargar modelos pre-entrenados
aria2c = 'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M'
%cd assets
hugging_user = "".join(["l", "j", "1995"])
huggingface_repo = f"https://huggingface.co/{hugging_user}/{v + c + w}/resolve/main/"
!{aria2c} {huggingface_repo}hubert_base.pt -d hubert -o hubert_base.pt
!{aria2c} {huggingface_repo}rmvpe.pt -d rmvpe -o rmvpe.pt
!{aria2c} {huggingface_repo}uvr5_weights/onnx_dereverb_By_FoxJoy/vocals.onnx -d uvr5_weights/onnx_dereverb_By_FoxJoy -o vocals.onnx
for model in ['D', 'G', 'f0D', 'f0G']:
  !{aria2c} {huggingface_repo}pretrained_v2/{model}48k.pth -d pretrained_v2 -o {model}48k.pth
for model in [
        "HP2-%E4%BA%BA%E5%A3%B0vocals%2B%E9%9D%9E%E4%BA%BA%E5%A3%B0instrumentals",
        "HP2_all_vocals",
        "HP3_all_vocals",
        "HP5-%E4%B8%BB%E6%97%8B%E5%BE%8B%E4%BA%BA%E5%A3%B0vocals%2B%E5%85%B6%E4%BB%96instrumentals",
        "HP5_only_main_vocal",
        "VR-DeEchoAggressive",
        "VR-DeEchoDeReverb",
        "VR-DeEchoNormal",
]:
  !{aria2c} {huggingface_repo}uvr5_weights/{model}.pth -d uvr5_weights -o {model}.pth
%cd ..

!rm -r /content/sample_data
!mkdir -p dataset/{raw,split,clean}

from IPython.display import clear_output
clear_output(wait=True)
print("\n---------------------------\n| Dependencias instaladas |\n---------------------------\n")

In [ ]:
# @title Conectar con Google Drive (Opcional)

from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# @title Entrenar modelo

NOMBRE_DEL_MODELO = "xadii"  # @param {type:"string"}
NUMERO_DE_PASOS = 1000 # @param {type:"integer"}
# @markdown Cada cuantos pasos se deberia guardar el modelo en disco en caso de fallos:
GUARDAR_PASOS = 100 # @param {type:"integer"}
SAMPLE_RATE = 48000
NUMERO_DE_HILOS = 8  # @param {type:"integer"}
ALGORITMO = "harvest"  # @param [ "pm", "dio", "harvest" ]
BATCHSIZE = 32  # @param {type:"slider", min:0, max:40, step:1}
# @markdown Guardar cache del entrenamiento en vram (solo recomendado para entrenamientos con menos de 10 minutos de audio)
CACHE_EN_VRAM = False  # @param {type:"boolean"}
# @markdown Solamente guardar el ultimo ckpt para ahorrar espacio en disco
ULTIMO_CKPT = True  # @param {type:"boolean"}

# aliases de variables de español a ingles
MODELNAME = NOMBRE_DEL_MODELO
MODELEPOCH = NUMERO_DE_PASOS
EPOCHSAVE = GUARDAR_PASOS
THREADCOUNT = NUMERO_DE_HILOS
ALGO = ALGORITMO

# convertir variables del form para ser leibles por los scripts
MODELSAMPLE = str(SAMPLE_RATE // 1000) + "k"
CACHEDATA = int(CACHE_EN_VRAM)
ONLYLATEST = int(ULTIMO_CKPT)

!python3 trainset_preprocess_pipeline_print.py dataset/clean {BITRATE} {THREADCOUNT} logs/{MODELNAME} True
!python3 extract_f0_print.py logs/{MODELNAME} {THREADCOUNT} {ALGO}
!python3 extract_feature_print.py cpu 1 0 0 logs/{MODELNAME}
!python3 train_nsf_sim_cache_sid_load_pretrain.py -e {MODELNAME} -sr {MODELSAMPLE} -f0 1 -bs {BATCHSIZE} -g 0 -te {MODELEPOCH} -se {EPOCHSAVE} -pg pretrained/f0G{MODELSAMPLE}.pth -pd pretrained/f0D{MODELSAMPLE}.pth -l {ONLYLATEST} -c {CACHEDATA}